In [2]:
import os
import conda

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

from mpl_toolkits.basemap import Basemap

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pygrib
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.collections import PolyCollection
import time

def print_time(t0):
    t1 = time.time()
    return t1-t0

In [4]:
# starting and ending viewazimuths
azim_start = 200
azim_end = 200

# starting and ending view elevations
elev_start = 30
elev_end = 30

# starting and ending zooms
dist_start = 250
dist_end = 250

# starting and ending center latitude
lat_start = 50
lat_end = 50

# starting and ending longitude
lon_start = -80
lon_end = -80

# starting and ending layer separation
delev_start = 5
delev_end = 5

# delta center degrees
# latitude window for center
# tradoff with dist
ddeg = 1

#save files setup
savepath = '../plots/map/movie/animation/'
filename = 'stage3_'


num_steps = 300

t0 = time.time()

variables = ['Wind speed (gust)','Visibility','Maximum/Composite radar reflectivity','Total Cloud Cover']
colors = ['viridis','magma','jet','viridis']
ranges = [(0,10),(0,90000),(-10,50),(0,100)]

for i in range(num_steps):
    delev = delev_start-i*((delev_start-delev_end)/(num_steps-1))
    hour = str(i-24*(i//24)).zfill(2)
    day = str(1+i//24).zfill(2)
    try:
        grbs = pygrib.open(f'../data/weather/august_full/rap_130_201808{day}_{hour}00_000.grb2')
    except:
        pass
    
    datas = []
    elev = np.arange(len(variables))*delev

    for variable in variables:
        grb = grbs.select(name=variable)[0]
        lats, lons = grb.latlons()
        datas.append(grb['values'])

    map = Basemap(llcrnrlon=-360,llcrnrlat=-90,urcrnrlon=360,urcrnrlat=90,)

    fig = plt.figure(figsize=[10,10],dpi=150)
    ax = Axes3D(fig)

    ax.add_collection3d(map.drawcoastlines(linewidth=0.25))
    ax.add_collection3d(map.drawcountries(linewidth=0.35))

    ax.set_axis_off()
    ax.azim = azim_start-i*((azim_start-azim_end)/(num_steps-1))
    ax.elev = elev_start-i*((elev_start-elev_end)/(num_steps-1))
    ax.dist = dist_start-i*((dist_start-dist_end)/(num_steps-1))
    
    center_lat = lat_start-i*((lat_start-lat_end)/(num_steps-1))
    center_lon = lon_start-i*((lon_start-lon_end)/(num_steps-1))


    for ix,data in enumerate(datas):
        # fourth dimention - colormap
        # create colormap according to x-value (can use any 50x50 array)
        color_dimension = data # change to desired fourth dimension
        minn, maxx = color_dimension.min(), color_dimension.max()
        norm = matplotlib.colors.Normalize(minn, maxx)
        m = plt.cm.ScalarMappable(norm=norm, cmap=colors[ix])
        m.set_array([])
        fcolors = m.to_rgba(color_dimension)

        ax.plot_surface(lons,lats,np.ones(lats.shape)*elev[ix]+delev/10,cstride=5,rstride=5,facecolors=fcolors, vmin=ranges[ix][0], vmax=ranges[ix][0], shade=False)

    ax.set_xlim3d(center_lon-ddeg,center_lon+ddeg)
    ax.set_ylim3d(center_lat-ddeg,center_lat+ddeg)
    if i < 10:
        fig.savefig(f'{savepath}{filename}0000{i}.png', dpi=fig.dpi)
    elif i >=10 and i < 100:
        fig.savefig(f'{savepath}{filename}000{i}.png', dpi=fig.dpi)
    elif i >=100 and i < 1000:
        fig.savefig(f'{savepath}{filename}00{i}.png', dpi=fig.dpi)
    elif i > 1000:
        fig.savefig(f'{savepath}{filename}0{i}.png', dpi=fig.dpi)
    plt.close()
    print(f'Printed {filename}0000{i}.png in',print_time(t0),'seconds')
    t0 = time.time()

Printed stage3_00000.png in 15.293901920318604 seconds
Printed stage3_00001.png in 15.139692068099976 seconds
Printed stage3_00002.png in 17.820783138275146 seconds
Printed stage3_00003.png in 18.21835708618164 seconds
Printed stage3_00004.png in 17.13685131072998 seconds
Printed stage3_00005.png in 15.931737184524536 seconds
Printed stage3_00006.png in 15.299463987350464 seconds
Printed stage3_00007.png in 15.178648948669434 seconds
Printed stage3_00008.png in 15.773086071014404 seconds
Printed stage3_00009.png in 15.20265793800354 seconds
Printed stage3_000010.png in 15.806948184967041 seconds
Printed stage3_000011.png in 15.504855871200562 seconds
Printed stage3_000012.png in 14.744480609893799 seconds
Printed stage3_000013.png in 14.788241147994995 seconds
Printed stage3_000014.png in 17.49171209335327 seconds
Printed stage3_000015.png in 17.65628409385681 seconds
Printed stage3_000016.png in 17.825826168060303 seconds
Printed stage3_000017.png in 18.509485006332397 seconds
Printed